In [2]:
import pandas as pd
import glob
import time

path = '../data/raw/*.csv'
csv_files = glob.glob(path)
df = []

for file in csv_files:
    df = pd.read_csv(file)
    print(list(df))

['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']


In [5]:
def shift_column(df, col):
    col_list = list(df)
    #col1_index = df.columns.get_loc(col1) #or list(df.columns).index(col1)
    #col2_index = df.columns.get_loc(col2)
    #col_list[col1_index], col_list[col2_index] = col_list[col2_index], col_list[col1_index]
    #df.columns = col_list
    col1 = df.pop(col)
    df.insert(len(col_list) - 1, col, col1)

def numbers_only(string):
    year_index = string.index(' y')
    year = string[:year_index]
    duration = 0
    if ' months' in string:
        month_index_start = string.index('s ') + 2
        month_index_end = string.index(' m')
        month = string[month_index_start:month_index_end]
        month_adjusted = str(float(month) / 12)
        duration = float(year + month_adjusted)
    else:
        duration = year
    return duration 

i = 0

for file in csv_files:
    i += 1
    df = pd.read_csv(file)
    if 'remaining_lease' in df.columns:
        shift_column(df, 'remaining_lease') #idt need to define a separate function
        #print(df1[:5])
     #df = pd.concat([df, df1]) #I realised I forgot to exclude header when concatenating...

#print(df[:5])
        
    print(len(df))
    df.drop_duplicates()
    print(len(df))
    
    start = time.time()
    df['month'] = df['month'].apply(lambda x: pd.to_datetime(x + '-01'))
    df.sort_values(by='month')

    elapsed = time.time() - start
    print(f'took {elapsed} seconds to run')

    if 'remaining_lease' in df.columns:
        try:
            df['remaining_lease'].astype('int32')
        except ValueError:
            df['remaining_lease'].apply(lambda x: numbers_only(x) if type(x) == str else None)

    for column in df.columns:
        exclude = ['month', 'floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease']
        if column not in exclude:
            df[column] = df[column].str.replace('-', ' ').str.lower()
            df[column] = df[column].astype('category')

    i_string = str(i)
    df.to_csv(f'../data/cleaned data {i_string}.csv', index=False)

52203
52203
took 33.86044144630432 seconds to run
369651
369651
took 211.72824263572693 seconds to run
195062
195062
took 107.92504262924194 seconds to run
37153
37153
took 19.941157817840576 seconds to run
287196
287196
took 156.9709141254425 seconds to run
